In [4]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
peliculas = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TP1/movies.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#P5. Para las películas que duran más de tres horas, ¿cuál es la película con más ROI?
df_limpio = peliculas[ ['title','runtime','revenue','budget'] ] #Eliminamos columnas innecesarias.
df_limpio = df_limpio[ df_limpio['runtime'] > 180]  #Filtramos por duracion mayor a 3 horas.
df_limpio = df_limpio.loc[~((df_limpio['budget'].astype(int) == 0) | (df_limpio['revenue'].astype(int) == 0))].dropna() #Eliminamos ceros y NaN.
df_limpio['ROI'] = (df_limpio['revenue'] - df_limpio['budget'].astype(int)) / df_limpio['budget'].astype(int) #Calculamos ROI.
df_limpio.loc[[df_limpio['ROI'].idxmax()]] #Buscamos la película com mayor ROI.

In [ ]:
#P20. ¿Cuál es la probabilidad de que una película de un género en particular tenga una calificación promedio mayor a 7? Obtenga las probabilidades de cada género posible.
df_limpio = peliculas[ ['title','genres','vote_average'] ].dropna()
df_limpio['Rating mayor a 7'] = df_limpio['vote_average'].map(lambda x: 1 if x > 7 else 0) #Creo columna, 1 si rating>7, 0 si rating <=7.
df_limpio.genres = df_limpio.genres.map(lambda x: x.split(',')) #Separo los generos de c/ peli en una lista.
df_limpio = df_limpio.explode('genres')
df_limpio.groupby(['genres']).mean().rename(columns={'Rating mayor a 7':'probabilidad'}).drop(columns=['vote_average'])

In [ ]:
#C3. Para variables continuas existe la entropía diferencial, un análogo al caso discreto. Para las productoras con más de 50 películas, ¿Cúal es la 
#productora con más y menos entropía diferencial para su ROI (Return of Investment)? ¿Qué interpretación puede hacer de esto?
from scipy.stats import differential_entropy

df_limpio = peliculas[ ['title','revenue','budget','production_companies'] ].dropna()
df_limpio.production_companies = df_limpio.production_companies.map(lambda x: x.split(',')) #Separo las companias de c/ peli en una lista.
df_limpio = df_limpio.explode('production_companies')

df_limpio = df_limpio.loc[~((df_limpio['budget'].astype(int) == 0) | (df_limpio['revenue'].astype(int) == 0))].dropna() #Eliminamos ceros y NaN.
df_limpio['ROI'] = (df_limpio['revenue'] - df_limpio['budget'].astype(int)) / df_limpio['budget'].astype(int) #Calculamos ROI.

#df_limpio = df_limpio.groupby('production_companies').filter(lambda x : len(x)>50)
df_limpio = df_limpio[df_limpio.groupby('production_companies').production_companies.transform('count')>50].copy()

ROI_companias = df_limpio.groupby(['production_companies'], sort=False)['ROI'].apply(list).reset_index(name='ROI') #Agrupo todos los ROI de cada compania.
ROI_companias['entropia'] = ROI_companias.ROI.map(lambda x: differential_entropy(x)) #Calculo entropía.
ROI_companias['entropia'].loc[[ROI_companias['entropia'].idxmax()]]
ROI_companias['entropia'].loc[[ROI_companias['entropia'].idxmin()]]


19    1.770665
Name: entropia, dtype: float64

In [ ]:
#V4. 

In [ ]:
#N6. Queremos predecir el género de la película según su descripción. Para esto vamos a buscar para una descripción nueva los 10 vecinos más cercanos 
#y asignar el género que más se repita entre ellos. Pruebe este algoritmo para 5 películas al azar y para una descripción inventada que se le ocurra 
#para una película nueva de Disney (su corrector valorará si logra hacerlo reír (? ).
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

df_limpio = peliculas[ ['title','genres','overview'] ].dropna().reset_index()
overview_mas_largo = df_limpio.overview.map(lambda x: len(x)).max()

stopwordsEng = set(stopwords.words('english'))
countVec = CountVectorizer(lowercase = True, stop_words = stopwordsEng, max_features = overview_mas_largo)
buscaVecinos = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
matrizVectores = countVec.fit_transform(df_limpio.overview)
buscaVecinos.fit(matrizVectores)

def vecinos_pelicula(pelicula):
  busqueda = countVec.transform(pelicula.overview)
  distancias, indices = buscaVecinos.kneighbors(busqueda, n_neighbors = 10)

  df_vecinos = df_limpio.loc[df_limpio.index[indices[0]]]
  df_vecinos.genres = df_vecinos.genres.map(lambda x: x.split(','))
  df_vecinos = df_vecinos.explode('genres')

  generos = df_vecinos.genres.mode().tolist()
  print("\nLos géneros de", pelicula.title.values[0], "son:", pelicula.genres.values[0])
  print("Los generos mas recurrentes de las 10 pelis vecinas son:", generos)

for i in range(5):
  vecinos_pelicula(df_limpio.sample()) #Obtenemos una peli random.

rey_leon = {'title': ['The Lion King'], 'genres': ['Animation,Drama,Musical'], 'overview': ["Prince's uncle kills father so he enlists the help of two immature vagrants to exact revenge. He falls in love and marries his cousin after he kills his uncle, taking his place as king."]}
rey_leon_df = pd.DataFrame(data=rey_leon)
vecinos_pelicula(rey_leon_df)



In [8]:
#P36. Consiga una matriz cuyas columnas sean los cast id y crew ids (diferenciados para no colisionar), que tenga una fila por cada película 
#y que los valores sean la cantidad de roles para los que esa persona participa en la película. Estime cuánto ocuparía en RAM teniendo en cuenta 
#que cada número pesa 4 bytes, y en caso de no entrar construyala como matriz dispersa.

cast = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TP1/cast.csv")
cast_limpio = cast[ ['id', 'cast_id'] ].dropna()
crew = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TP1/crew.csv")
crew_limpio = crew[ ['id', 'crew_id'] ].dropna()


cast_limpio = cast_limpio.groupby(['cast_id', 'id'], sort=False).size().to_frame().reset_index()
cast_limpio.columns = ['cast_id', 'id', 'roles_quantity']
ids = len(cast_limpio.id.unique())
cast_ids = len(cast_limpio.cast_id.unique())
bytes = ids * cast_ids * 4 #35474019376 bytes = 35Gb

#pd.pivot_table(cast_limpio, index='id', columns='cast_id', values='roles_quantity', fill_value=0, sort=False)






#asd[ asd[0] == 5 ]
#cast[cast["cast_id"] == 1242792]
#1242792


#asd = cast_limpio.groupby(['cast_id', 'id'], sort=False).size().idxmax()
#cast[cast["cast_id"] == asd[0]]

#matriz = pd.pivot_table(companias.explode('original_language'), index='name', columns='original_language', values='original_language', aggfunc='count', fill_value=0, sort=False)

35474019376

In [1]:
#C7. Para todas las productoras obtenga una matriz cuadrada en donde la posición i,j sea la divergencia de kullback leibler entre la distribución de 
#idiomas originales de la productora i respecto de la productora j. Grafique con plt.imshow.
from scipy.stats import entropy

companias = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TP1/companies.csv").dropna()

pelis_limpio = peliculas[ ['title','original_language','production_companies'] ].dropna()
pelis_limpio.production_companies = pelis_limpio.production_companies.map(lambda x: x.split(','))
pelis_limpio = pelis_limpio.explode('production_companies')

idiomas_companias = pelis_limpio.groupby('production_companies', sort=False)['original_language'].unique().to_frame().reset_index().rename(columns = {'production_companies':'id'})
idiomas_companias.id = pd.to_numeric(idiomas_companias.id)

companias = pd.merge(companias, idiomas_companias)

matriz = pd.pivot_table(companias.explode('original_language'), index='name', columns='original_language', values='original_language', aggfunc='count', fill_value=0, sort=False)
matriz = matriz.div(matriz.sum(axis=1), axis=0).agg(list, axis=1).to_frame().reset_index()
matriz.columns = ['name', 'language_distribution']

res = pd.DataFrame(columns = companias.name, index = companias.name)
res



NameError: ignored